In [158]:
# Descarga tu repositorio desde GitHub
!git clone https://github.com/Wmarbar/CProgram_incidencias.git

# Cambia de directorio para acceder a los archivos
%cd CProgram_incidencias

# Verifica que los archivos estén
!ls


Cloning into 'CProgram_incidencias'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 66 (delta 36), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 1.65 MiB | 17.84 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias
 constructor4_proyecto_II.ipynb   README.md
 input				 'Trabajo-CalidadDeDatos Ver4-2.pdf'
 logo_u.png


# __PROGRAMACIÓN PARA ANALÍTICA DE DATOS__

<img src="https://github.com/WMARBAR/CProgram_incidencias/blob/main/logo_u.png?raw=1" alt="Logo Universidad Central" width="300" />

> ## __PROYECTO: Estimación Área bajo la curva__

> ## Profesor: Jorge Victorino [jvictorinog@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Estudiante: Sergio Andrés Sánchez Cárdenas [correo@ucentral.edu.co](mailto:correo@ucentral.edu.co)<br>
> ## Estudiante: Wilson Felipe Mártinez Barrantes [wmartinezb1@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Grupo: 5 <br>
> ### Facultad de Ingeniería y Ciencias Básicas <br>
> ### Universidad Central <br>
> <br>

---


# FUNCTIONS

In [159]:
import pandas as pd
import numpy as np
from datetime import datetime

def load_excel(path):

 df_incidentes = pd.read_excel(path)
 return df_incidentes


def delta_dates(df, col1, col2):
    try:
        # Verificar si las columnas existen
        if col1 not in df.columns or col2 not in df.columns:
            print(f"Las columnas '{col1}' o '{col2}' no existen en el DataFrame.")
            return df

        # Convertir columnas a datetime con coerción
        df[col1] = pd.to_datetime(df[col1], errors='coerce')
        df[col2] = pd.to_datetime(df[col2], errors='coerce')

        # Avisar si hay muchas fechas inválidas (pero continuar)
        if df[col1].isna().any():
            print(f"Algunos valores en '{col1}' no son fechas válidas y se marcarán como NaT.")
        if df[col2].isna().any():
            print(f"Algunos valores en '{col2}' no son fechas válidas y se marcarán como NaT.")

        # Calcular diferencia en días (donde ambas fechas sean válidas)
        nueva_col = f"{col1}_{col2}_days"
        df[nueva_col] = (df[col2] - df[col1]).dt.days

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df

def key_generator(df, columnas):
    try:
        # Validar columnas
        for col in columnas:
            if col not in df.columns:
                print(f"La columna '{col}' no existe en el DataFrame.")
                return df

        # Nombre de la nueva columna
        nombre_key = '_'.join(columnas) + '_KEY'

        # Función de limpieza para cada valor
        def limpiar_valor(v):
            if isinstance(v, float) and v.is_integer():
                return str(int(v))
            return str(v)

        # Aplicar limpieza y concatenar
        df[nombre_key] = df[columnas].apply(lambda row: '_'.join(limpiar_valor(v) for v in row), axis=1)

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df


def drop_nan_by_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df_limpio = df.dropna(subset=[col])
    return df_limpio


def deduplicar_incidentes(df, key_col='Incidente_Estructura_KEY', dias_col='Creacion_Inicio_days'):
    if key_col not in df.columns or dias_col not in df.columns:
        print(f"Las columnas '{key_col}' o '{dias_col}' no existen.")
        return df, pd.DataFrame()

    indices_a_conservar = []

    # Función para aplicar por grupo
    def resolver_grupo(grupo):
        sin_cero = grupo[grupo[dias_col] != 0]
        if not sin_cero.empty:
            idx = sin_cero.index[0]
        else:
            idx = grupo.index[0]
        indices_a_conservar.append(idx)
        return

    # Agrupar y marcar qué índices se deben conservar
    df.groupby(key_col, group_keys=False).apply(resolver_grupo)

    # Separar los que se quedan y los que se van
    df_filtrado = df.loc[indices_a_conservar].reset_index(drop=True)
    df_duplits = df.drop(index=indices_a_conservar).reset_index(drop=True)

    print(f"Registros conservados: {len(df_filtrado)} | Registros duplicados removidos: {len(df_duplits)}")

    return df_filtrado, df_duplits


def unique_validator(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return

    duplicados = df.duplicated(subset=[col]).sum()

    if duplicados > 0:
        print(f"Hay {duplicados} valores duplicados en la columna '{col}'.")
    else:
        print(f"Todos los valores en la columna '{col}' son únicos.")



def nan_detector_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df[f'{col}_NAN_DETECTOR'] = df[col].apply(lambda x: 1 if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else 0)

    return df

def deduplicar_estructuras_por_vereda(df, key_col='Estructura', prioridad_col='Vereda'):
    # Validar que las columnas existan
    if key_col not in df.columns or prioridad_col not in df.columns:
        print(f"Las columnas '{key_col}' o '{prioridad_col}' no existen.")
        return df

    # Función para aplicar por grupo
    def resolver_grupo(grupo):
        con_vereda = grupo[grupo[prioridad_col].apply(lambda x: isinstance(x, str) and x.strip() != '') | grupo[prioridad_col].notna()]
        if not con_vereda.empty:
            return con_vereda.iloc[[0]]  # Uno cualquiera con vereda
        else:
            return grupo.iloc[[0]]  # Si no hay vereda, igual uno cualquiera

    # Aplicar la lógica por grupo
    df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)

    return df_filtrado


def limpiar_nombres_columnas(df):
    df.columns = df.columns.str.strip().str.replace(r'\s+', '_', regex=True)
    return df

def corregir_basura_columna(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    def _fix_encoding(x):
        if isinstance(x, str):
            try:
                return x.encode('latin1').decode('utf-8')
            except Exception:
                return x
        return x

    df[col] = df[col].apply(_fix_encoding)
    return df

def merge_excluir_llave(df1, df2, llave_df1, llave_df2):
    # Verificar existencia de las columnas
    if llave_df1 not in df1.columns or llave_df2 not in df2.columns:
        print(f"Las llaves '{llave_df1}' o '{llave_df2}' no existen.")
        return df1

    # Hacer el merge completo primero
    df_merged = df1.merge(df2, left_on=llave_df1, right_on=llave_df2, how='left')

    # Eliminar la llave de df2 si es distinta a la de df1
    if llave_df1 != llave_df2:
        df_merged = df_merged.drop(columns=[llave_df2])

    return df_merged

def guardar_excel(df, nombre_archivo='salida.xlsx', sheet_name='Hoja1'):
    try:
        df.to_excel(nombre_archivo, index=False, sheet_name=sheet_name)
        print(f"Archivo guardado exitosamente como '{nombre_archivo}'.")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")


def marcar_estado_interes(df, col='Estado'):
    palabras_clave = ['GPRE', 'PRER', 'TAMB', 'GEAM', 'AMPO', 'GERE']
    # Usamos regex para buscar cualquiera de las palabras
    patron = '|'.join(palabras_clave)

    df['estado_interes'] = df[col].astype(str).str.contains(patron).astype(int)

    return df

def ones_selector(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df_filtrado = df[df[col] == 1].copy()
    return df_filtrado

def spliter(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    # Separar por espacio
    nuevas_cols = df[col].astype(str).str.split(expand=True)

    # Renombrar columnas nuevas con sufijos: col_1, col_2, etc.
    nuevas_cols.columns = [f"{col}_{i+1}" for i in range(nuevas_cols.shape[1])]

    # Unir al DataFrame original
    df_expandido = pd.concat([df, nuevas_cols], axis=1)

    return df_expandido

def contar_columnas_con_dato(df, columnas):
    # Validar que las columnas existan
    for col in columnas:
        if col not in df.columns:
            print(f"La columna '{col}' no existe en el DataFrame.")
            return df

    # Contar columnas con dato no nulo y no vacío
    df['Estados'] = df[columnas].apply(
        lambda fila: sum(1 for x in fila if pd.notna(x) and (not isinstance(x, str) or x.strip() != '')),
        axis=1
    )

    return df

def conteo_duplicados_por_llave(df, llave):
    if llave not in df.columns:
        print(f"La columna '{llave}' no existe en el DataFrame.")
        return pd.DataFrame()

    # Contar ocurrencias por cada valor único de la llave
    conteo = df[llave].value_counts().reset_index()
    conteo.columns = [llave, 'Repeticiones']

    return conteo

def limpiar_espacios_en_columnas(df, columnas):
    for col in columnas:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: "" if isinstance(x, str) and x.strip() == "" else x)
        else:
            print(f"La columna '{col}' no existe en el DataFrame.")
    return df

def prioridadchanger(df, col_prioridad='Prioridad', col_referencia='Inicio_Cierre_days'):
    # Mapeo de prioridad a días
    mapeo_dias = {
        'Semana': 7,
        'Mes': 30,
        'Trimestre': 90,
        'Semestre': 180,
        'Año': 365,
        'Dos años': 730,
        'Tres años': 1095,
        'Seis años': 2190
    }

    # Calcular mediana para los valores NaN
    mediana_referencia = df[col_referencia].median()

    # Crear nueva columna con los días
    df['prioridad_days'] = df[col_prioridad].map(mapeo_dias)

    # Rellenar NaN con la mediana de la duración real
    df['prioridad_days'] = df['prioridad_days'].fillna(mediana_referencia)

    return df


def numeric_feats(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe.")
        return

    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"La columna '{col}' no es numérica.")
        return

    serie = df[col]

    resumen = {
        'Columna': col,
        'Valores únicos': serie.nunique(),
        'Nulos': serie.isna().sum(),
        'Máximo': serie.max(),
        'Mínimo': serie.min(),
        'Más frecuente': serie.mode().iloc[0] if not serie.mode().empty else None,
        'Frecuencia del más frecuente': serie.value_counts().iloc[0] if not serie.mode().empty else None,
        'Media': serie.mean(),
        'Mediana': serie.median(),
        'Desviación estándar': serie.std()
    }

    return pd.DataFrame([resumen])




def evaluar_prioridad_cumplida(df, prioridad_col='prioridad_days',
                                cierre_col='Creacion_Cierre_days',
                                inicio_cierre_col='Inicio_Cierre_days',
                                fecha_creacion_col='Creacion',
                                fecha_inicio_col='Inicio'):
    # Validar columnas
    columnas = [prioridad_col, cierre_col, inicio_cierre_col, fecha_creacion_col, fecha_inicio_col]
    for col in columnas:
        if col not in df.columns:
            print(f"La columna '{col}' no existe.")
            return df

    # Convertir fechas a datetime
    df[fecha_creacion_col] = pd.to_datetime(df[fecha_creacion_col], errors='coerce')
    df[fecha_inicio_col] = pd.to_datetime(df[fecha_inicio_col], errors='coerce')

    # Calcular días hasta hoy
    hoy = pd.to_datetime(datetime.now().date())
    df['dias_desde_creacion'] = (hoy - df[fecha_creacion_col]).dt.days
    df['dias_desde_inicio'] = (hoy - df[fecha_inicio_col]).dt.days

    # Rellenar NaNs en los días reales con días hasta hoy
    df[cierre_col] = df[cierre_col].fillna(df['dias_desde_creacion'])
    df[inicio_cierre_col] = df[inicio_cierre_col].fillna(df['dias_desde_inicio'])

    # Calcular delta contra prioridad
    df['delta_creacion_cierre_vs_prioridad'] = df[cierre_col] - df[prioridad_col]
    df['delta_inicio_cierre_vs_prioridad'] = df[inicio_cierre_col] - df[prioridad_col]

    # Evaluación: ¿se cumplió la prioridad?
    df['cumplimiento_creacion'] = df['delta_creacion_cierre_vs_prioridad'].apply(
        lambda x: 'Cumplida' if x <= 0 else 'Incumplida'
    )

    df['cumplimiento_inicio'] = df['delta_inicio_cierre_vs_prioridad'].apply(
        lambda x: 'Cumplida' if x <= 0 else 'Incumplida'
    )

    return df



def categoc_features(df, col):
    """
    Genera un resumen estadístico de una variable categórica:
    - Total por categoría
    - Porcentaje sobre el total
    - Porcentaje acumulado estilo Pareto

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        col (str): Nombre de la columna categórica.

    Returns:
        pd.DataFrame: Resumen con columnas ['Total', 'Porcentaje', '%_Acumulado']
    """
    if col not in df.columns:
        print(f"❌ La columna '{col}' no existe en el DataFrame.")
        return pd.DataFrame()

    total = df[col].notna().sum()
    conteo = df[col].value_counts(dropna=True).sort_values(ascending=False)
    porcentaje = (conteo / total * 100).round(2)
    pareto = porcentaje.cumsum().round(2)

    resumen = pd.DataFrame({
        'Total': conteo,
        'Porcentaje': porcentaje,
        '%_Acumulado': pareto
    }).reset_index().rename(columns={'index': col})

    return resumen




# CARGA DE ARCHIVOS

In [160]:

df_incidentes = load_excel("./input/incidentes1.xlsx")
df_incidentes_original=load_excel("./input/incidentes1.xlsx")
df_estructura = load_excel("./input/estructura1.xlsx")

# 1. TRATAMIENTO DATA INCIDENCIAS

## 1.1. ARREGLO DE LA DATA INCIDENCIAS

In [161]:

#1. Marcar estado:
df_incidentes=marcar_estado_interes(df_incidentes,'Estado')
#2. Seleccionar estados de interes:
df_incidentes=ones_selector(df_incidentes,'estado_interes')
# df_incidentes es la base de datos que se editará

#3. Reservar data original con nulls y duplicados para comparar despues:
df_incidentes_original=marcar_estado_interes(df_incidentes_original,'Estado')
df_incidentes_original=ones_selector(df_incidentes_original,'estado_interes')

## 1.2. TRATAMIENTO DE df_incidentes (Base de datos con la cantidad de incidencias registradas)

In [162]:
#1. Calcular la diferencia de dias entre las fechas de Creacion, Inicio y Cierre de la incidencia
df_incidentes=delta_dates(df_incidentes,'Creacion','Inicio')
df_incidentes=delta_dates(df_incidentes,'Creacion','Cierre')
df_incidentes=delta_dates(df_incidentes,'Inicio','Cierre')
#2. Crear una llave para asegurar que cada Incidente tenga una estructura unica asignada
df_incidentes=key_generator(df_incidentes,['Incidente','Estructura'])
#3. Eliminar NaN de estructuras ya que no puede haber una incidencia sin estructura
df_incidentes=drop_nan_by_column(df_incidentes,'Estructura')
#4. Eliminar duplicados de La columna incidentes en base a la columna Incidente_Estructura_KEY (llave entre incidente y estructura)
# Adicional se crea la base de datos df_incidentes_dups que contiene los duplicados que se eliminaron.
df_incidentes, df_incidentes_dups =deduplicar_incidentes(df_incidentes)
#5. La fecha de cierre tiene NaNs que se podrian traducir como incidencias no cerradas por lo que se deben marcar para calcularlas
df_incidentes=nan_detector_column(df_incidentes,'Cierre')
#6. Garantizar que las columnas no tengan espacios incomodos
df_incidentes=limpiar_nombres_columnas(df_incidentes)
#7. Garantizar una columna contadora para facilitar el analisis
df_incidentes['incidentes_counter']=1


Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.
Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.
Registros conservados: 3118 | Registros duplicados removidos: 1398


<ipython-input-159-5e831ccaf802>:92: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(key_col, group_keys=False).apply(resolver_grupo)


## 1.3. ANALISIS DE INCIDENTES DUPLICADOS

In [163]:
base_nodup=len(df_incidentes) #df_incidentes es la base arreglada con incidencias unicas
base_dup=len(df_incidentes_dups) #df_incidentes_dups fue lo que se eliminó de df_incidentes
conteo_duplicados_por_llave(df_incidentes_original,'Incidente')# Aqui se hace un breve analisis de la data sin tratar

,Incidente,Repeticiones
0,2985102,10
1,2561196,8
2,2516568,8
3,2418967,8
4,2549859,8
...,...,...
3133,1618956,1
3134,1749655,1
3135,1643965,1
3136,1749630,1


In [164]:
print(f'TOTAL DE LA BASE (con duplicados) : {base_nodup+base_dup}')
print(f'TOTAL DE LA BASE (sin duplicados) : {base_nodup}')
print(f'REGISTROS UNICOS : {base_nodup}')
print(f'REGISTROS DUPLICADOS : {base_dup}')
print(f'% DUPLICADOS : {round((base_dup / (base_nodup + base_dup)) * 100, 2)}%')


TOTAL DE LA BASE (con duplicados) : 4516
TOTAL DE LA BASE (sin duplicados) : 3118
REGISTROS UNICOS : 3118
REGISTROS DUPLICADOS : 1398
% DUPLICADOS : 30.96%


## 1.4. ANALISIS DE NANS

In [165]:
# Datos Nulls en el df sin arreglar
df_incidentes_original.isna().sum()

,0
Incidente,0
Estructura,24
Tipo incidente,0
Estado,0
Prioridad,23
Creacion,0
Inicio,0
Cierre,1288
estado_interes,0


In [166]:
# Datos Nulls en el df Arreglado
df_incidentes.isna().sum()

,0
Incidente,0
Estructura,0
Tipo_incidente,0
Estado,0
Prioridad,17
Creacion,0
Inicio,0
Cierre,743
estado_interes,0
Creacion_Inicio_days,0


## 1.5. TIPOS DE DATOS DTYPES

In [167]:
# dtypes del df arreglado
df_incidentes['Estructura']=df_incidentes['Estructura'].astype(int)
df_incidentes.dtypes

,0
Incidente,int64
Estructura,int64
Tipo_incidente,object
Estado,object
Prioridad,object
Creacion,datetime64[ns]
Inicio,datetime64[ns]
Cierre,datetime64[ns]
estado_interes,int64
Creacion_Inicio_days,int64


# 2. Base de datos de las Estructuras (Contiene informacion sociodemografica de las estructuras que puede ser analizadas con las insidencias)

## 2.1. ARREGLO DE LA DATA ESTRUCTURAS

In [168]:
# 1.) Arreglar las columnas de las estructuras
df_estructura= limpiar_nombres_columnas(df_estructura)
# 2.) Me quedo solo con las estructuras que tengan
df_estructura= drop_nan_by_column(df_estructura,'Estructura')
# 3.) Eliminiar duplicados por id de Estructura y dejando aquellas que tengan la vereda
df_estructura= deduplicar_estructuras_por_vereda(df_estructura)
# 4.) Tratamiento de las palabras de la columna Depto
df_estructura = corregir_basura_columna(df_estructura, 'Depto')
df_estructura['Depto'] = df_estructura['Depto'].replace('NARIÃ‘O', 'NARIÑO')
# 5.) Tratamiento de las palabras de la columna Municipio
df_estructura = corregir_basura_columna(df_estructura, 'Municipio')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('IBAGUÃ‰', 'IBAGUÉ')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('JERUSALÃ‰N', 'JERUSALÉN')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('GUACHENÃ‰', 'GUACHENÉ')
# 6.) Tratamiento de las cordenadas
df_estructura['Latitud'] = df_estructura['Latitud'].replace(',', '.')
df_estructura['Longitud'] = df_estructura['Longitud'].replace(',', '.')

<ipython-input-159-5e831ccaf802>:141: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)


In [169]:
df_incidentes.columns

Index(['Incidente', 'Estructura', 'Tipo_incidente', 'Estado', 'Prioridad',
       'Creacion', 'Inicio', 'Cierre', 'estado_interes',
       'Creacion_Inicio_days', 'Creacion_Cierre_days', 'Inicio_Cierre_days',
       'Incidente_Estructura_KEY', 'Cierre_NAN_DETECTOR',
       'incidentes_counter'],
      dtype='object')

# 3. Base de incidencias completa con la informacion de estructuras

## 3.1. Merge de df_incidentes y df_estructura

In [170]:
df_merged= merge_excluir_llave(df_incidentes,df_estructura,'Estructura','Estructura')


## 3.2. Calculo del estado

In [171]:
df_merged=spliter(df_merged,'Estado')
df_merged=contar_columnas_con_dato(df_merged, ['Estado_1', 'Estado_2', 'Estado_3','Estado_4','Estado_5','Estado_6','Estado_7'])
df_merged['Estructura']=df_merged['Estructura'].astype(int)
df_merged = prioridadchanger(df_merged, 'Prioridad', 'Inicio_Cierre_days')
df_merged.dtypes



,0
Incidente,int64
Estructura,int64
Tipo_incidente,object
Estado,object
Prioridad,object
Creacion,datetime64[ns]
Inicio,datetime64[ns]
Cierre,datetime64[ns]
estado_interes,int64
Creacion_Inicio_days,int64


## 3.3.VARIABLES CATEGORICAS

In [172]:
df_merged=limpiar_espacios_en_columnas(df_merged, ['Tipo_incidente', 'Estado', 'Prioridad',
                                                   'Depto', 'Municipio', 'Vereda', 'Cia',
                                                   'Estado_1', 'Estado_2', 'Estado_3','Estado_4',
                                                   'Estado_5','Estado_6','Estado_7'])

### 3.3.1. VARIABLE MUNICIPIO

In [184]:
categoc_features(df_merged, 'Municipio')

,Municipio,Total,Porcentaje,%_Acumulado
0,SANTA MARÍA,164,5.29,5.29
1,UBALÁ,163,5.26,10.55
2,POPAYÁN,84,2.71,13.26
3,JAMUNDÍ,80,2.58,15.84
4,GUASCA,71,2.29,18.13
...,...,...,...,...
146,GUACHENÉ,1,0.03,99.82
147,MERCADERES,1,0.03,99.85
148,NOBSA,1,0.03,99.88
149,CAMPOHERMOSO,1,0.03,99.91


### 3.3.2. VARIABLE Tipo_incidente

In [185]:
categoc_features(df_merged, 'Tipo_incidente')

,Tipo_incidente,Total,Porcentaje,%_Acumulado
0,VEGETACIÓN,1499,48.08,48.08
1,CONSTRUCCIONES,1165,37.36,85.44
2,OBRAS,236,7.57,93.01
3,PERMISO INGRESO,125,4.01,97.02
4,INVASIÓN/EXPLANACION,61,1.96,98.98
5,INVASIÓN/EXPLANACION,32,1.03,100.01


### 3.3.3. VARIABLE Prioridad

In [186]:
categoc_features(df_merged, 'Prioridad')

,Prioridad,Total,Porcentaje,%_Acumulado
0,Año,1111,35.83,35.83
1,Tres años,467,15.06,50.89
2,Seis años,393,12.67,63.56
3,Semestre,356,11.48,75.04
4,Dos años,337,10.87,85.91
5,Trimestre,283,9.13,95.04
6,Mes,150,4.84,99.88
7,Semana,4,0.13,100.01


## 3.4. VARIABLES NUMERICAS

In [174]:
#Funcion que me entrega valores de tiempo de solución o tiempo en mora
df_merged=evaluar_prioridad_cumplida(df_merged)

### 3.4.1. Variable Creacion_Inicio_days: Diferencia entre la fecha de creación y el inicio de atención del incidente

In [175]:
numeric_feats(df_merged,'Creacion_Inicio_days')


,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,Creacion_Inicio_days,425,0,4270,0,0,2136,126.075369,0.0,463.011275


### 3.4.2. Variable Creacion_Cierre_days: Días transcurridos entre la creación y el cierre del incidente

In [176]:
numeric_feats(df_merged,'Creacion_Cierre_days')


,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,Creacion_Cierre_days,1327,0,6520.0,0.0,6467.0,77,1332.510904,573.5,1600.174994


### 3.4.3. Variable Inicio_Cierre_days: Días transcurridos entre el inicio de atención y el cierre del incidente

In [177]:
numeric_feats(df_merged,'Inicio_Cierre_days')


,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,Inicio_Cierre_days,1298,0,6520.0,0.0,6467.0,66,1206.435536,482.0,1525.887254


### 3.4.4. Variable prioridad_days: Tiempo estimado para resolver un incidente según su prioridad asignada

In [178]:
numeric_feats(df_merged,'prioridad_days')


,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,prioridad_days,9,0,2190.0,7.0,365.0,1111,680.997434,365.0,656.229857


### 3.4.5. Variable delta_creacion_cierre_vs_prioridad: Diferencia entre días reales y días esperados según la prioridad

In [179]:

numeric_feats(df_merged,'delta_creacion_cierre_vs_prioridad')

,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,delta_creacion_cierre_vs_prioridad,1453,0,6142.0,-2060.0,4277.0,55,651.51347,137.0,1259.480825


### 3.4.6. Variable delta_inicio_cierre_vs_prioridad: Diferencia entre días reales y días esperados según la prioridad

In [180]:
numeric_feats(df_merged,'delta_inicio_cierre_vs_prioridad')

,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,delta_inicio_cierre_vs_prioridad,1541,0,6142.0,-2190.0,4277.0,49,525.438101,89.0,1239.802257


### 3.4.7. Variable Estados: Conteo de estados reportados para cada incidente

In [181]:
numeric_feats(df_merged,'Estados')

,Columna,Valores únicos,Nulos,Máximo,Mínimo,Más frecuente,Frecuencia del más frecuente,Media,Mediana,Desviación estándar
0,Estados,6,0,7,2,2,1775,2.557409,2.0,0.754247


# EXPORTACION DE LAS BASES

In [182]:
guardar_excel(df_merged)

Archivo guardado exitosamente como 'salida.xlsx'.


In [183]:
from google.colab import files
files.download('salida.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>